In [2]:
import nest_asyncio
from IPython.display import Markdown, display

from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.core import PromptTemplate
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex, ServiceContext, SimpleDirectoryReader, StorageContext
from llama_index.core.postprocessor import SentenceTransformerRerank
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import Settings
import qdrant_client

/home/henry/Documents/Projects/data-science/chat/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# allows nested access to the event loop
nest_asyncio.apply()

In [4]:
#docs path
input_dir_path = './docs'

In [ ]:
# Function to create an index from documents
collection_name = "chat_with_docs"
client = qdrant_client.QdrantClient(
    host="localhost",
    port=6333,
)
def create_index(documents):
    vector_store = QdrantVectorStore(client=client, collection_name=collection_name)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    index = VectorStoreIndex.from_documents(
        documents,
        storage_context=storage_context,
    )
    return index

In [ ]:
# Setup LLM and embedding model and reranker
llm=Ollama(model="llama3.2:1b", request_timeout=120.0)
embed_model = HuggingFaceEmbedding( model_name="BAAI/bge-large-en-v1.5", trust_remote_code=True)
rerank = SentenceTransformerRerank(
    model="cross-encoder/ms-marco-MiniLM-L-2-v2", top_n=3
)

In [14]:
# Load data from specified directory
loader = SimpleDirectoryReader(
    input_dir=input_dir_path,
    required_exts=[".pdf"],
    recursive=True
)
docs = loader.load_data()

# Check number of documents loaded
print(f"Number of documents loaded: {len(docs)}")

# Creating an index over loaded data
Settings.embed_model = embed_model

try:
    index = create_index(docs)
    print('Using Qdrant collections')
    print(f"Index contains {len(index)} documents.")
except Exception as e:
    print(f"Error creating index: {e}")
    index = VectorStoreIndex.from_documents(docs, show_progress=True)

# Create the query engine with reranking enabled
Settings.llm = llm
query_engine = index.as_query_engine(
    similarity_top_k=10, 
    node_postprocessors=[rerank]
)

# ====== Customize prompt template ======
qa_prompt_tmpl_str = (
    "Context information is below.\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n"
    "Given the context information above I want you to think step by step to answer the query in a crisp manner, in case you don't know the answer say 'I don't know!'.\n"
    "Query: {query_str}\n"
    "Answer: "
)
qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)

query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

# Generate the response for a specific query
response = query_engine.query("What exactly is DSPy?")
print("Response:", response)

# Optionally display the response in Markdown format (if using Jupyter Notebook)
display(Markdown(str(response)))

Number of documents loaded: 32
Error creating index: [Errno -2] Name or service not known


Generating embeddings: 100%|██████████| 45/45 [01:47<00:00,  2.38s/it]


Response: Based on the provided text, it appears that DSPy (Deep Sense Prompting and Yield) is a programming model developed by Stanford Natural Language Processing Group. It allows developers to define natural language signatures for task-specific prompting and abstracts the process of prompting Large Language Models (LMs). This enables the creation of self-improving and pipeline-adaptive prompts, which can be finetuned using these signatures.


Based on the provided text, it appears that DSPy (Deep Sense Prompting and Yield) is a programming model developed by Stanford Natural Language Processing Group. It allows developers to define natural language signatures for task-specific prompting and abstracts the process of prompting Large Language Models (LMs). This enables the creation of self-improving and pipeline-adaptive prompts, which can be finetuned using these signatures.